# <u>Muestreo, Normalización y Estandarización</u>

### Importación de librerías necesarias

In [ ]:
# Importar librerias necesarias
import numpy as np
import pandas as pd

import scipy as sp
import scipy.stats as st
import statsmodels.api as sm 

import matplotlib.pyplot as plt

### Carga de datos

In [ ]:
# Importar datos en formato xls.
data_hipotecas = pd.read_excel('data/hipotecas.xlsx')

In [ ]:
# hallazgos iniciales
print('dimensiones',data_hipotecas.shape)
print('Nro de tiendas:', len(np.unique(data_hipotecas.TIENDA)))
print('Nro de vendedores:', len(np.unique(data_hipotecas.ID_VENDEDOR)))

In [ ]:
data_hipotecas.head()

## Técnicas de selección de elementos por muestreo

### 1. Muestreo aleatorio simple:

De acuerdo con el principio de igual probabilidad, se seleccionan aleatoriamente n muestras de la población.

Escenario aplicable: todos los individuos de la muestra son igualmente probables (uniformes).

Según los escenarios aplicables, el muestreo aleatorio se divide en muestreo con reemplazo y muestreo sin reemplazo.

In [ ]:
# Muestreo aleatorio simple por cantidad
print('Data completa=',data_hipotecas.shape)
print('Data MAS=',data_hipotecas.sample(n=50, replace=False).shape)

In [ ]:
data_sample = data_hipotecas.sample(n=50, replace=False)

In [ ]:
data_sample['TIENDA'].value_counts().sort_index()

In [ ]:
data_hipotecas['TIENDA'].value_counts().sort_index()

In [ ]:
data_sample['TIENDA'].value_counts(normalize = True).sort_index()

In [ ]:
data_hipotecas['TIENDA'].value_counts(normalize = True).sort_index()

In [ ]:
# Muestreo aleatorio simple por porcentaje
print('Data completa=',data_hipotecas.shape)
print('Data MAS=',data_hipotecas.sample(frac = 0.1, replace=False).shape)

In [ ]:
data_sample_frac = data_hipotecas.sample(frac = 0.1, replace=False)

In [ ]:
data_sample_frac['TIENDA'].value_counts().sort_index()

In [ ]:
data_hipotecas['TIENDA'].value_counts().sort_index()

In [ ]:
data_sample_frac['TIENDA'].value_counts(normalize = True).sort_index()

In [ ]:
data_hipotecas['TIENDA'].value_counts(normalize = True).sort_index()

### 2. Muestreo estratificado:

Divida todas las muestras individuales en varias categorías de acuerdo con ciertas características y luego aplique un muestreo aleatorio o un muestreo equidistante de cada categoría para seleccionar individuos para formar una muestra.

Ventajas: Puede reducir significativamente los errores de muestreo y facilitar la investigación por separado para diferentes tipos de muestras de datos.

Escenarios aplicables: datos con características como atributos y etiquetas de lógica de clasificación.

In [ ]:
# Muestreo Estratificado por cantidad
df_estrato = data_hipotecas.groupby('TIENDA', group_keys=False).apply(lambda x: x.sample(n = 200, replace = False))

In [ ]:
df_estrato['TIENDA'].value_counts().sort_index()

In [ ]:
# Muestreo Estratificado por proporción
df_estrato2 = data_hipotecas.groupby('TIENDA', group_keys=False).apply(lambda x: x.sample(frac = 0.1, replace = False))

In [ ]:
df_estrato2['TIENDA'].value_counts().sort_index()

## Pruebas de Normalidad

In [ ]:
# Importar datos en formato xls.
churners_data = pd.read_excel('data/Churners.xlsx')

In [ ]:
churners_data.head()

In [ ]:
churners_data.groupby(['CHURN'])['ID'].count()

In [ ]:
churners_data_vol=churners_data[churners_data['CHURN']=='Voluntario']

In [ ]:
churners_data_vol.head()

Seleccionamos la variable de interés:

In [ ]:
churners_ing=churners_data_vol['INGRESO']

In [ ]:
churners_ing.head()

### 1. Gráfico de Histograma
Un gráfico simple y de uso común para verificar rápidamente la distribución de una muestra de datos es el histograma. En el histograma, los datos se dividen en un número predeterminado de grupos llamados bins. Luego, los datos se clasifican en cada contenedor y se retiene el recuento del número de observaciones en cada contenedor.

El gráfico muestra los contenedores en el eje x manteniendo su relación ordinal, y el recuento en cada contenedor en el eje y. Una muestra de datos tiene una distribución gaussiana del diagrama del histograma, que muestra la forma familiar de campana.

In [ ]:
# Análisis gráfico
fig = plt.figure(figsize=(10,4))
plt.hist(churners_ing,bins=50,color='blue')
plt.title('Distribución de ingresos',fontsize=20)
plt.ylabel('Frecuencia',fontsize=20)

plt.grid()
plt.show()

### 2. Gráfico de cuantiles-cuantiles
Otro gráfico popular para verificar la distribución de una muestra de datos es el gráfico de cuantiles-cuantiles, gráfico QQ o gráfico QQ para abreviar. Esta gráfica genera su propia muestra de la distribución idealizada con la que estamos comparando, en este caso la distribución gaussiana. Las muestras idealizadas se dividen en grupos (por ejemplo, 5), llamados cuantiles. Cada punto de datos de la muestra se empareja con un miembro similar de la distribución idealizada en la misma distribución acumulativa.

Los puntos resultantes se trazan como un diagrama de dispersión con el valor idealizado en el eje xy la muestra de datos en el eje y. Una línea de puntos en un ángulo de 45 grados desde la parte inferior izquierda del gráfico hasta la parte superior derecha mostrará una coincidencia perfecta para la distribución. A menudo, se traza una línea en la trama para ayudar a aclarar esta expectativa. Las desviaciones de los puntos de la línea muestran una desviación de la distribución esperada.

In [ ]:
sm.qqplot(churners_ing, line='s') 
plt.show()

### Pruebas de normalidad estadística
Hay muchas pruebas estadísticas que podemos utilizar para cuantificar si una muestra de datos parece extraída de una distribución gaussiana. Cada prueba hace diferentes suposiciones y considera diferentes aspectos de los datos.

En esta sección, analizaremos 3 pruebas de uso común que puede aplicar a sus propias muestras de datos.

Interpretación de una prueba
Cada prueba devolverá al menos dos cosas:

* **Estadística :** una cantidad calculada por la prueba que se puede interpretar en el contexto de la prueba comparándola con los valores críticos de la distribución de la estadística de prueba.
* **valor p :** se utiliza para interpretar la prueba, en este caso si la muestra se extrajo de una distribución gaussiana.
Las pruebas suponen que la muestra se extrajo de una distribución gaussiana. Técnicamente, esto se llama hipótesis nula o H0. Se elige un nivel de umbral llamado alfa, típicamente 5% (o 0.05), que se usa para interpretar el valor p.

En la implementación de SciPy de estas pruebas, puede interpretar el valor p de la siguiente manera.
* p <= alfa : rechazar H0, no es normal.
* p> alpha : falla al rechazar H0, normal.

Un resultado superior al 5% no significa que la hipótesis nula sea cierta. Significa que es muy probable que sea cierto dada la evidencia disponible. El valor p no es la probabilidad de que los datos se ajusten a una distribución gaussiana; se puede considerar como un valor que nos ayuda a interpretar la prueba estadística.

### 3. Prueba de Shapiro-Wilk
La prueba de Shapiro-Wilk evalúa una muestra de datos y cuantifica la probabilidad de que los datos se extraigan de una distribución gaussiana, llamada así por Samuel Shapiro y Martin Wilk.

En la práctica, se cree que la prueba de Shapiro-Wilk es una prueba confiable de normalidad, aunque se sugiere que la prueba puede ser adecuada para muestras de datos más pequeñas, por ejemplo, miles de observaciones o menos.

La función Shapiro () SciPy calculará el Shapiro-Wilk en un conjunto de datos dado. La función devuelve tanto el estadístico W calculado por la prueba como el valor p.

In [ ]:
shapiro_test = st.shapiro(churners_ing)
shapiro_test

In [ ]:
print('estadístico:',round(shapiro_test[0],3))
print('p_value:',round(shapiro_test[1],3))

In [ ]:
# normality test
stat, p = st.shapiro(churners_ing)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

### 4. Prueba de D'Agostino
La prueba K² de D'Agostino calcula estadísticas de resumen a partir de los datos, a saber, curtosis y asimetría, para determinar si la distribución de datos se aparta de la distribución normal, llamada así por Ralph D'Agostino.

* El sesgo es una cuantificación de cuánto se empuja una distribución hacia la izquierda o hacia la derecha, una medida de asimetría en la distribución.
* La curtosis cuantifica qué parte de la distribución hay en la cola. Es una prueba estadística simple y de uso común para la normalidad.

In [ ]:
# D'Agostino and Pearson's Test
from scipy.stats import normaltest

# normality test
stat, p = normaltest(churners_ing)
print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

### 5. Prueba de Anderson-Darling
La prueba de Anderson-Darling es una prueba estadística que se puede utilizar para evaluar si una muestra de datos proviene de una de las muchas muestras de datos conocidas, que lleva el nombre de Theodore Anderson y Donald Darling.

Se puede utilizar para comprobar si una muestra de datos es normal. La prueba es una versión modificada de una prueba estadística de bondad de ajuste no paramétrica más sofisticada llamada prueba de Kolmogorov-Smirnov.

Una característica de la prueba de Anderson-Darling es que devuelve una lista de valores críticos en lugar de un solo valor p. Esto puede proporcionar la base para una interpretación más completa del resultado.

La función anderson de SciPy implementa la prueba Anderson-Darling. Toma como parámetros la muestra de datos y el nombre de la distribución para probarla. De forma predeterminada, la prueba se comparará con la distribución gaussiana.

In [ ]:
# Anderson-Darling Test
from scipy.stats import anderson

# normality test
result = anderson(churners_ing)
print('Statistic: %.3f' % result.statistic)
p = 0

for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
    else:
        print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))

Podemos interpretar los resultados si no rechazamos la hipótesis nula de que los datos son normales si el estadístico de prueba calculado es menor que el valor crítico en un nivel de significancia elegido.

Podemos ver que en cada nivel de significancia, la prueba ha encontrado que los datos no siguen una distribución normal:

### 6. Prueba Kolmogorov-Smirnov

En estadística, la prueba de Kolmogórov-Smirnov (también prueba K-S) es una prueba no paramétrica que determina la bondad de ajuste de dos distribuciones de probabilidad entre sí.

La prueba de una muestra compara la distribución subyacente F (x) de una muestra con una distribución dada G (x). La prueba de dos muestras compara las distribuciones subyacentes de dos muestras independientes. Ambas pruebas son válidas solo para distribuciones continuas.

Conviene tener en cuenta que la prueba Kolmogórov-Smirnov es más sensible a los valores cercanos a la mediana que a los extremos de la distribución. La prueba de Anderson-Darling proporciona igual sensibilidad con valores extremos.

La hipótesis nula es que las dos distribuciones son idénticas, F (x) = G (x) para todo x; la alternativa es que no sean idénticos.

Para utilizar este test como prueba de normalidad, debemos antes estandarizar la data a probar:
                                `normed_data = (data - data.mean()) / data.std()`

In [ ]:
from scipy.stats import kstest

In [ ]:
# estandarizamos
churners_std = (churners_ing - churners_ing.mean()) / churners_ing.std()

# normality test
stat, p = kstest(churners_std,'norm')
print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

Se debe rechazar la hipótesis nula a favor de la hipótesis alternativa. Por lo que la distribución de x no se aproxima a la normal.

Hagamos la prueba con 10 mil números aleatorios de una distribución normal estándar con esta prueba:

In [ ]:
np.random.seed(123)
x = np.random.randn(10000)
kstest(x, 'norm')

No se puede rechazar la hipótesis nula a favor de la hipótesis alternativa. Por lo que la distribución de x se aproxima a la normal.

No olvidar estandarizar antes de hacer la prueba KS para normalidad para cualquier conjunto de datos.

In [ ]:
mu, sigma = 10, 5 # media y desvio estandar
normal = st.norm(mu, sigma)

x_test = normal.rvs(10000)

# estandarizamos
x_test_std = (x_test - x_test.mean()) / x_test.std()

# normality test
stat, p = kstest(x, 'norm')
print('Statistics=%.3f, p=%.3f' % (stat, p))

No se puede rechazar la hipótesis nula a favor de la hipótesis alternativa. Por lo que la distribución de x se aproxima a la normal.

Si queremos comparar la distribución para 2 muestras independientes con KS, podemos usar `ks_2samp`:

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
week1 = np.genfromtxt("data/week1.csv",  delimiter=",")
week2 = np.genfromtxt("data/week2.csv",  delimiter=",")

In [ ]:
plt.hist(week1, bins=20, alpha=0.5, color='b')
plt.hist(week2, bins=20,alpha=0.5, color='r')
leg = ['week1', 'week2']
plt.legend(leg)
plt.show()

In [ ]:
ks_2samp(week1,week2)

* Ho: la distribución de las 2 muestras son iguales
* Ha: la distribución de las 2 muestras son diferentes

Se debe rechazar la hipótesis nula a favor de la hipótesis alternativa. Por lo que la distribución de los datos son diferentes.

## Proceso de estandarización de datos

La estandarización de datos es un paso muy importante en el preprocesamiento de datos. En aplicaciones prácticas, a menudo nos encontramos con un conjunto de datos que contiene una variedad de características, a menudo con diferentes distribuciones e intervalos, con diferentes niveles (dimensión), que es fácil de afectar nuestra capacitación modelo.. La estandarización de datos es eliminar los efectos de la escala, la característica y las diferencias de distribución en el modelo.

Además, después de que toda la función esté estandarizada, podemos hacer carteras ponderadas para generar nuevos indicadores, y los datos en bruto a menudo no admiten que ponderemos directamente.

### 1. Estandarización z

La estandarización de la puntuación Z se estandariza en función de la normalización de la media y la desviación estándar de los datos originales, y su fórmula de transformación es:

**normalización = ( x – media ) / desviación típica**

Este método es adecuado para la mayoría de los tipos de datos, que es muy amplio. Desde la fórmula, podemos ver que la media se convertirá en 0 después de la transformación, y la variación y la diferencia estándar se convertirán en 1 (considerada la fórmula de la varianza), esta parte si no entiende, es posible que desee razonarlo.

In [ ]:
# estandarización de datos
score_churners = st.zscore(churners_ing) # estandarizamos los datos

In [ ]:
score_churners

### 2. Escalamiento MIN-MAX

El método de estandarización MIN-MAX transformará linealmente los datos originales, y su fórmula de conversión es:

**escalamiento = ( x – xmin ) / ( xmax – xmin )**

Obviamente, cuando X es el valor máximo, se convertirá a 1; cuando X es un valor mínimo, el nuevo valor correspondiente es 0; El conjunto completo de datos se distribuye dentro del intervalo de 0 a 1, y la distribución de datos. y no cambiará.

In [ ]:
# escalado de datos
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler_churners = scaler.fit_transform(np.reshape(np.array(churners_ing),(-1,1)))

In [ ]:
# Comparando resultados de variables generadas
a = pd.DataFrame(np.array(churners_ing))
b = pd.DataFrame(score_churners)
c = pd.DataFrame(scaler_churners)

variable = pd.concat([a,b,c],axis = 1)
variable.columns = ['churners_ing','score_churners','scaler_churners']
variable.head()

In [ ]:
variable.describe()

In [ ]:
print('churners promedio:',round(variable.churners_ing.mean(),2))
print('churners varianza:',round(variable.churners_ing.std(),4))

print('score promedio:',round(variable.score_churners.mean(),2))
print('score varianza:',round(variable.score_churners.std(),4))

print('escaler promedio:',round(variable.scaler_churners.mean(),2))
print('escaler varianza:',round(variable.scaler_churners.std(),4))